In [1]:
import importlib
from pydoc import render_doc
# ENVIRONMENT
%load_ext autoreload
%autoreload 2
import pandas as pd
import dotenv
import os

dotenv.load_dotenv('.env')
MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])


In [2]:
# nautilus_trader imports

from nautilus_trader.model.identifiers import Venue, InstrumentId
from nautilus_trader.model.data import Bar, BarType, QuoteTick
from nautilus_trader.config import BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.engine import BacktestResult
from nautilus_trader.trading.strategy import ImportableStrategyConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos, maybe_unix_nanos_to_dt, unix_nanos_to_dt
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.cache.cache import Cache
from nautilus_trader.model.position import Position
from nautilus_trader.model.objects import Price


from decimal import Decimal
from datetime import datetime

# other imports
from pandas import Timestamp
import importlib
import mplfinance as mpf
import matplotlib.pyplot as plt

# my packages
import indicators
from indicators import TrackerFloat, TrackerMulti

import strategies.bollinger_cluster
from strategies.bollinger_cluster import BollingerCluster
import data_utils
import utils
importlib.reload(indicators)
importlib.reload(strategies.bollinger_cluster)
importlib.reload(data_utils)
importlib.reload(utils)


catalog = ParquetDataCatalog(CATALOG_PATH)

start = dt_to_unix_nanos(pd.Timestamp("2023-11-01 00:00:00"))
end = start + pd.Timedelta(days=30).value 

venue = "SIM_EIGHTCAP"
instrument_id = f"EURUSD.{venue}"


In [3]:
venue_configs = [
    BacktestVenueConfig(
        name=venue,
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["10_000 USD"],
    ),
]

data_configs = [
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=QuoteTick,
        instrument_id=instrument_id,
        start_time=start,
        end_time=end,
    ),
]

strategies = [
    ImportableStrategyConfig(
        strategy_path="strategies.bollinger_cluster:BollingerCluster",
        config_path="strategies.bollinger_cluster:BollingerClusterConfig",
        config=dict(
            instrument_id=instrument_id,
            bar_type=f"{instrument_id}-15-MINUTE-BID-INTERNAL",
            bb_params=[
                (20, 3),
            ]
        ),
    ),
]

configs = [BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    data=data_configs,
    venues=venue_configs,
)]

node = BacktestNode(configs)

In [4]:
results = node.run()
res = results[0]
backtest_start = maybe_unix_nanos_to_dt(res.backtest_start)
backtest_end = maybe_unix_nanos_to_dt(res.backtest_end)


2023-12-17T15:32:19.850405001Z [INF] BACKTESTER-001.BacktestEngine: =================================================================
2023-12-17T15:32:19.850414001Z [INF] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2023-12-17T15:32:19.850416001Z [INF] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2023-12-17T15:32:19.850417001Z [INF] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2023. All rights reserved.
2023-12-17T15:32:19.850418001Z [INF] BACKTESTER-001.BacktestEngine: =================================================================
2023-12-17T15:32:19.850418002Z [INF] BACKTESTER-001.BacktestEngine: 
2023-12-17T15:32:19.850418003Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2023-12-17T15:32:19.850629001Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2023-12-17T15:32:19.850631001Z [INF] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2023-12-17T15:32:19.8506

2023-11-03T16:00:00.536000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


2023-11-03T16:00:00.536000000Z [INF] BACKTESTER-001.Portfolio: EURUSD.SIM_EIGHTCAP margin_maint=32.15 USD
2023-11-03T16:00:00.536000000Z [INF] BACKTESTER-001.Portfolio: Updated AccountState(account_id=SIM_EIGHTCAP-001, account_type=MARGIN, base_currency=USD, is_reported=False, balances=[AccountBalance(total=9_999.98 USD, locked=32.15 USD, free=9_967.83 USD)], margins=[MarginBalance(initial=0.00 USD, maintenance=32.15 USD, instrument_id=EURUSD.SIM_EIGHTCAP)], event_id=ec7c5579-dfe9-451b-8748-5cf9e17b15e7).
2023-11-03T16:00:00.536000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] PositionOpened(instrument_id=EURUSD.SIM_EIGHTCAP, position_id=SIM_EIGHTCAP-1-001, account_id=SIM_EIGHTCAP-001, opening_order_id=O-20231103-1600-001-000-31, closing_order_id=None, entry=SELL, side=SHORT, signed_qty=-1000.0, quantity=1_000, peak_qty=1_000, currency=USD, avg_px_open=1.07098, avg_px_close=0.0, realized_return=0.00000, realized_pnl=-0.02 USD, unrealized_pnl=0.00 USD, ts_opened=16990272005360

2023-11-07T09:00:00.157000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


2023-11-08T18:10:36.609000000Z [INF] BACKTESTER-001.Portfolio: EURUSD.SIM_EIGHTCAP margin_init=32.00 USD
2023-11-08T18:10:36.609000000Z [INF] BACKTESTER-001.Portfolio: Updated AccountState(account_id=SIM_EIGHTCAP-001, account_type=MARGIN, base_currency=USD, is_reported=False, balances=[AccountBalance(total=10_001.94 USD, locked=64.12 USD, free=9_937.82 USD)], margins=[MarginBalance(initial=32.00 USD, maintenance=32.12 USD, instrument_id=EURUSD.SIM_EIGHTCAP)], event_id=4952ddea-222e-4bc0-b40b-461d02b27512).
2023-11-08T18:10:36.609000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderFilled(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231107-0900-001-000-51, venue_order_id=SIM_EIGHTCAP-1-009, account_id=SIM_EIGHTCAP-001, trade_id=SIM_EIGHTCAP-1-004, position_id=SIM_EIGHTCAP-1-002, order_side=SELL, order_type=LIMIT, last_qty=1000, last_px=1.07108 USD, commission=0.02 USD, liquidity_side=MAKER, ts_event=1699467036609000000).
2023-11-08T18:10:36.609000000Z [INF] BACKTES

...

2023-11-16T02:00:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderInitialized(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231116-0200-001-000-88, side=SELL, type=LIMIT_IF_TOUCHED, quantity=1_000, time_in_force=GTD, post_only=False, reduce_only=False, quote_quantity=False, options={'price': '1.08541', 'trigger_price': '1.08541', 'trigger_type': 'DEFAULT', 'expire_time_ns': 1700100030000000000, 'display_qty': None}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=OTO, order_list_id=OL-20231116-0200-001-000-30, linked_order_ids=['O-20231116-0200-001-000-89', 'O-20231116-0200-001-000-90'], parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=ENTRY).
2023-11-16T02:00:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderInitialized(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231116-0200-001-000-89, side=BUY, type=STOP_MARKET, quantity=1_000, time_in_force=GTC

2023-11-22T08:30:00.241000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


2023-11-22T08:30:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderInitialized(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231122-0830-001-000-130, side=BUY, type=LIMIT_IF_TOUCHED, quantity=1_000, time_in_force=GTD, post_only=False, reduce_only=False, quote_quantity=False, options={'price': '1.08971', 'trigger_price': '1.08971', 'trigger_type': 'DEFAULT', 'expire_time_ns': 1700641830000000000, 'display_qty': None}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=OTO, order_list_id=OL-20231122-0830-001-000-44, linked_order_ids=['O-20231122-0830-001-000-131', 'O-20231122-0830-001-000-132'], parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=ENTRY).
2023-11-22T08:30:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderInitialized(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231122-0830-001-000-131, side=SELL, type=STOP_MARKET, quantity=1_000, time_in_force

2023-11-29T09:15:00.178000000Z [ERR] BACKTESTER-001.OrderMatchingEngine(SIM_EIGHTCAP): Cannot fill order: no fills from book when fills were expected (check sizes in data).


2023-11-29T09:15:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderInitialized(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231129-0915-001-000-175, side=BUY, type=LIMIT_IF_TOUCHED, quantity=1_000, time_in_force=GTD, post_only=False, reduce_only=False, quote_quantity=False, options={'price': '1.09897', 'trigger_price': '1.09897', 'trigger_type': 'DEFAULT', 'expire_time_ns': 1701249330000000000, 'display_qty': None}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=OTO, order_list_id=OL-20231129-0915-001-000-59, linked_order_ids=['O-20231129-0915-001-000-176', 'O-20231129-0915-001-000-177'], parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=ENTRY).
2023-11-29T09:15:00.000000000Z [INF] BACKTESTER-001.BollingerCluster-000: <--[EVT] OrderInitialized(instrument_id=EURUSD.SIM_EIGHTCAP, client_order_id=O-20231129-0915-001-000-176, side=SELL, type=STOP_MARKET, quantity=1_000, time_in_force

In [5]:
res

BacktestResult(trader_id='BACKTESTER-001', machine_id='Tobiass-MacBook-Air.local', run_config_id='c61ee6f4a68efa134d465094099578839d693630b0eeb092937436fa7cb9e6ee', instance_id='3b2c28fb-30b3-4717-a1b7-b75c6201ebc0', run_id='db24162f-1aba-4f33-96c7-7aa2818ddcd2', run_started=1702827143081309001, run_finished=1702827172782988001, backtest_start=1698796800065000000, backtest_end=1701388799630000000, elapsed_time=2591999.565, iterations=0, total_events=402, total_orders=180, total_positions=9, stats_pnls={'USD': {'PnL (total)': -3.3, 'PnL% (total)': -0.03299999999999272, 'Max Winner': 0.98, 'Avg Winner': 0.9671428571428571, 'Min Winner': 0.96, 'Min Loser': -5.03, 'Avg Loser': -5.035, 'Max Loser': -5.04, 'Expectancy': -0.3666666666666666, 'Win Rate': 0.7777777777777778}}, stats_returns={'Returns Volatility (252 days)': 0.021628433866714352, 'Average (Return)': -0.0002941455265749269, 'Average Loss (Return)': -0.004605478647490804, 'Average Win (Return)': 0.0009376639365438952, 'Sharpe Rati

In [6]:
print(res.backtest_start, dt_to_unix_nanos(start))
print(res.backtest_end, dt_to_unix_nanos(end))
engine = node.get_engine(res.run_config_id)
strategy: BollingerCluster = engine.trader.strategies()[0]
cache: Cache = strategy.cache

1698796800065000000 1698796800000000000
1701388799630000000 1701388800000000000


In [17]:
import mplfinance as mpf
import mplcursors

bars: list[Bar] = cache.bars(strategy.bar_type)

# filter for bars that are single_price
#bars = [b for b in bars if not b.is_single_price()]
# convert bars to dataframe without the dates without bars

df = utils.df_from_bars(bars)

#df = utils.remove_weekends(df) 

fig = mpf.figure(style="charles", figsize=(16,9), tight_layout=True)
ax = fig.add_subplot(2,1,1)
mpf.plot(df, ax=ax, type="candle")


/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


In [525]:
print(len(bars))
df = utils.df_from_bars(bars)
print(len(df))
df = utils.remove_weekends(df)
print(len(df))

2111
2111
2107


In [8]:
from nautilus_trader.model.data import BarSpecification
from nautilus_trader.model.objects import Money
from nautilus_trader.core.rust.model import AggregationSource, OrderSide
from bokeh.plotting import figure, curdoc, output_notebook, reset_output
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import ColumnDataSource, DatetimePicker, Button, TextInput, DatetimeTickFormatter
from bokeh.layouts import column, row
from bokeh.models import HoverTool

def handler1(some_value):
    print(some_value)
    pass

# create line withs enum
from enum import Enum
class LineWidth:
    THIN = 2
    MEDIUM = 4
    THICK = 9


def add_overlay_indicator_scatter_to_plot(p: figure, indicator: TrackerMulti, color: str = "black", **kwargs) -> figure:
    df = indicator.get_df()
    
    for i, col in enumerate(df.columns):
        p.scatter(df.index, df[col], color=color, **kwargs)
    
    return p

def add_overlay_indicator_to_plot(p: figure, indicator: TrackerMulti, colors: list[str] | None = None, line_widths: list[int] | None = None, **kwargs) -> figure:
    df = indicator.get_df()
    df = utils.remove_weekends(df)
    
    
    if colors is None:
            colors = ["black"] * len(df.columns)
    
    if line_widths is None:
        line_widths = [LineWidth.THIN] * len(df.columns)
    
    for i, col in enumerate(df.columns):
        p.line(df.index, df[col], color=colors[i], line_width=line_widths[i] ,**kwargs)
    return p

def add_positions_to_plot(p, positions: list[Position]):
    df =  pd.DataFrame([p.to_dict() for p in positions])
    
    # only use relevant columns to avoid warnings about "too large int values" from bokeh
    df = df[["ts_opened", "avg_px_open", "ts_closed", "avg_px_close", "realized_pnl", "peak_qty"]]
    
    df["ts_opened"] = df["ts_opened"].apply(lambda x: maybe_unix_nanos_to_dt(x))
    df["ts_closed"] = df["ts_closed"].apply(lambda x: maybe_unix_nanos_to_dt(x))
    
    renderers = [
        p.segment("ts_opened", "avg_px_open", "ts_closed", "avg_px_close", source=df, color="pink"),
    ]
    
    p.circle("ts_opened", "avg_px_open", source=df, color="blue"),
    p.circle("ts_closed", "avg_px_close", source=df, color="pink"),
    
    # add pnl hover tooltip
    p.add_tools(HoverTool(tooltips=[("realized_pnl", "@realized_pnl"),
                                    ("peak_qty", "@peak_qty")], renderers=renderers, mode="vline"))
    
    return p

    
def add_bars_to_plot(p, bars: list[Bar]):
    # empty bars
    if len(bars) == 0:
        return p
    
    bar_df = utils.df_from_bars(bars)
    
    print(bar_df.index)
    print(bar_df.columns)
    print(bar_df.head())
    
    #return p
    
    #p.xaxis.major_label_overrides = {
    #    i: date.strftime('%b %d') for i, date in enumerate(pd.to_datetime(bar_df.index))
    #}
    
    #p.xaxis.major_label_overrides = {
     #   i: ts.strftime('%Y-%m-%d %H:%S') for i, ts in enumerate(bar_df.index)
    #}
    
    # bar_df = utils.remove_weekends(bar_df)
    
    # ohlc bars
    inc = bar_df.close > bar_df.open
    dec = bar_df.open > bar_df.close
    
    bar_type: BarType = bars[0].bar_type
    bar_spec: BarSpecification = bar_type.spec
    
    width_ms = int(bar_spec.timedelta.total_seconds() * 1000 / 2)
    p.segment(bar_df.index, bar_df.high, bar_df.index, bar_df.low, color="black")
    p.vbar(bar_df.index[inc], width=width_ms, top=bar_df.open[inc], bottom=bar_df.close[inc], fill_color="#D5E1DD", line_color="black")
    p.vbar(bar_df.index[dec], width=width_ms, top=bar_df.open[dec], bottom=bar_df.close[dec], fill_color="#F2583E", line_color="black")

    return p

def get_layout(bars: list[Bar], sync_axis=None, picker_start=None, picker_end=None):
    
    p = figure(x_axis_type="datetime")
    
    p = add_bars_to_plot(p, bars)
    
    #bar_df = utils.df_from_bars(bars)
    #bar_df = utils.remove_weekends(bar_df)
    #source = ColumnDataSource(df)
    #p.xaxis.formatter = DatetimeTickFormatter(days="%d %b %Y")
    
    # ohlc bars
    #inc = bar_df.close > bar_df.open
    #dec = bar_df.open > bar_df.close
    
    #bar_type: BarType = bars[0].bar_type
    #bar_spec: BarSpecification = bar_type.spec
    
    #width_ms = int(bar_spec.timedelta.total_seconds() * 1000 / 2)
    #p.segment("date", "high", "date", "low", color="black", source=source)
    #p.vbar(bar_df.index[inc], width=width_ms, top=bar_df.open[inc], bottom=bar_df.close[inc], fill_color="#D5E1DD", line_color="black")
    #p.vbar(bar_df.index[dec], width=width_ms, top=bar_df.open[dec], bottom=bar_df.close[dec], fill_color="#F2583E", line_color="black")
    
    #p = add_bars_to_plot(p, bars)
    
    #p = add_overlay_indicator_scatter_to_plot(p, strategy.trackers[0])
    
    p = add_positions_to_plot(p, cache.positions())
    #p = add_overlay_indicator_to_plot(p, strategy.trackers[0])
    
    # map dataframe indices to date strings and use as label overrides
    #p.xaxis.major_label_overrides = {
    #    i: ts for i, ts in enumerate(utils.df_from_bars(bars).index)
    #}
    
    text1 = TextInput(title="title", value='value')
    picker_start = DatetimePicker(title="Start", value=picker_start)
    picker_end = DatetimePicker(title="End", value=picker_end, max_date=datetime.now())
    button = Button(label="button", button_type="success")
    button.on_click(lambda: handler1("click"))
    
    layout = column(
        row(text1, button),
        row(picker_start, picker_end),
        p
    )
    
    return layout
    


In [9]:
# plot prepared data
bars = cache.bars(strategy.bar_type)
print(len(bars))
bars = [b for b in bars if not b.is_single_price()]
print(len(bars))
layout = get_layout(bars, picker_start=backtest_start, picker_end=backtest_end)

# standalone HTML file
reset_output()
show(layout)

2879
2111
DatetimeIndex(['2023-11-30 23:45:00+00:00', '2023-11-30 23:30:00+00:00',
               '2023-11-30 23:15:00+00:00', '2023-11-30 23:00:00+00:00',
               '2023-11-30 22:45:00+00:00', '2023-11-30 22:30:00+00:00',
               '2023-11-30 22:15:00+00:00', '2023-11-30 22:00:00+00:00',
               '2023-11-30 21:45:00+00:00', '2023-11-30 21:30:00+00:00',
               ...
               '2023-11-01 02:30:00+00:00', '2023-11-01 02:15:00+00:00',
               '2023-11-01 02:00:00+00:00', '2023-11-01 01:45:00+00:00',
               '2023-11-01 01:30:00+00:00', '2023-11-01 01:15:00+00:00',
               '2023-11-01 01:00:00+00:00', '2023-11-01 00:45:00+00:00',
               '2023-11-01 00:30:00+00:00', '2023-11-01 00:15:00+00:00'],
              dtype='datetime64[ns, UTC]', name='date', length=2111, freq=None)
Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')
                              open     high      low    close       volume
date               

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [13]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib qt
import pandas as pd

In [14]:
df = utils.df_from_bars(bars)
#df = utils.remove_weekends(df)

import mplfinance as mpf

mpf.plot(df, type="candle", style="yahoo", volume=True, figratio=(16,9), figscale=1.5)


/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


pandas._libs.tslibs.timestamps.Timestamp